In [1]:
import semantic_kernel as sk

kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

api_key, org_id = sk.openai_settings_from_dot_env()
service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-0125", api_key=api_key, org_id=org_id),
)

plugin = kernel.import_plugin_from_prompt_directory("./Plugins/", "ReportPlugin")
OutlineJSON_function = plugin["OutlineJSON"]

OutlineJSON = await kernel.invoke(OutlineJSON_function, sk.KernelArguments(input="Báo cáo thị trường Tiêu dùng năm 2023"))
print(OutlineJSON)


{'report': {'title': 'Báo cáo thị trường Tiêu dùng năm 2023','introduction': 'Báo cáo này nhằm đánh giá tình hình thị trường tiêu dùng trong năm 2023, bao gồm xu hướng tiêu dùng, thị trường cạnh tranh, và ảnh hưởng của các yếu tố kinh tế - xã hội.','body': {'headings': ['1. Xu hướng tiêu dùng','2. Thị trường cạnh tranh','3. Ảnh hưởng của yếu tố kinh tế - xã hội']}, 'conclusion': 'Dựa trên các phân tích trong báo cáo, có thể kết luận rằng thị trường tiêu dùng năm 2023 đang phát triển mạnh mẽ và đầy tiềm năng.','recommendations': 'Dựa vào kết luận, chúng tôi đề xuất các biện pháp cần thiết để tận dụng cơ hội và đối phó với thách thức trong thị trường tiêu dùng năm 2023.'}}


In [2]:
from Plugins.utils import json_split
outline_dict = json_split(str(OutlineJSON))
outline_dict

{'title': 'Báo cáo thị trường Tiêu dùng năm 2023',
 'introduction': 'Báo cáo này nhằm đánh giá tình hình thị trường tiêu dùng trong năm 2023, bao gồm xu hướng tiêu dùng, thị trường cạnh tranh, và ảnh hưởng của các yếu tố kinh tế - xã hội.',
 'body': {'headings': ['1. Xu hướng tiêu dùng',
   '2. Thị trường cạnh tranh',
   '3. Ảnh hưởng của yếu tố kinh tế - xã hội']},
 'conclusion': 'Dựa trên các phân tích trong báo cáo, có thể kết luận rằng thị trường tiêu dùng năm 2023 đang phát triển mạnh mẽ và đầy tiềm năng.',
 'recommendations': 'Dựa vào kết luận, chúng tôi đề xuất các biện pháp cần thiết để tận dụng cơ hội và đối phó với thách thức trong thị trường tiêu dùng năm 2023.'}

In [3]:
Introduction_function = plugin["Introduction"]
Introduction = await kernel.invoke(Introduction_function, sk.KernelArguments(introduction=outline_dict["introduction"]
                                                                            , reportLayout = str(OutlineJSON)))
print(Introduction)

Báo cáo thị trường Tiêu dùng năm 2023 nhằm đánh giá và phân tích chi tiết về tình hình tiêu dùng trong năm, tập trung vào các yếu tố quan trọng như xu hướng tiêu dùng, thị trường cạnh tranh và ảnh hưởng của các yếu tố kinh tế - xã hội. Bằng việc đánh giá các thông tin và dữ liệu được cung cấp, báo cáo sẽ giúp độc giả hiểu rõ hơn về thị trường tiêu dùng năm 2023, từ đó đưa ra những nhận định và đề xuất cần thiết để phát triển và tận dụng cơ hội trong môi trường kinh doanh hiện nay.


In [4]:
from Plugins.google_search import GoogleSearchPlugin
from Plugins.utils import token_count
import asyncio
TOKEN_LIMIT = 8000

google_plugin = kernel.import_plugin_from_object(GoogleSearchPlugin(), plugin_name = "GoogleSearchPlugin")
google_search_function = google_plugin["GoogleSearch"]
get_content_function = google_plugin["GetContentFromURL"]

column_function = plugin["Column"]
column_summ_function = plugin["ColumnSumm"]
headings = outline_dict["body"]["headings"]

async def columnist(heading, title):
    search_result = []
    tokens = 0
    search_result_urls = await kernel.invoke(google_search_function, sk.KernelArguments(keyword = heading + f"{title}", num_results = 15))
    search_result_urls = search_result_urls.value
    while True:
        if not search_result_urls:
            break
        url = search_result_urls.pop(0)
        content = await kernel.invoke(get_content_function, sk.KernelArguments(url = url))
        content = content.value
        new_tokens = token_count(content)
        if tokens + new_tokens > TOKEN_LIMIT:
            print(tokens)
            if tokens >= TOKEN_LIMIT//2:
                break
            else:
                continue
        tokens += new_tokens
        search_result.append(content)
    search_result = " ".join(search_result)
    column = await kernel.invoke(column_function, sk.KernelArguments(input = search_result, body_content = heading))
    column_summ = await kernel.invoke(column_summ_function, sk.KernelArguments(input = search_result, body_content = heading))
    return str(column), str(column_summ)

coros = []
for heading in headings:
    coros.append(columnist(heading, outline_dict["title"]))

values = await asyncio.gather(*coros)
print(values)




0
4728
3975
3975
6825
0
0
7243
[('Báo cáo thị trường tiêu dùng Việt Nam năm 2023: 9 nguyên nhân thúc đẩy sự tăng trưởng của Việt Nam trong thập kỷ tới\n\nBáo cáo thị trường tiêu dùng Việt Nam năm 2023 đã đưa ra những nhận định quan trọng về xu hướng và kỳ vọng của người tiêu dùng Việt Nam trong thập kỷ tới. Dưới đây là 9 nguyên nhân chính thúc đẩy sự tăng trưởng của Việt Nam trong lĩnh vực tiêu dùng:\n\n1. Dân số người đi làm, dân số lao động cao và tỷ lệ nhóm phụ thuộc thấp: Việt Nam sở hữu nguồn lực lao động lớn, với tỷ lệ người đi làm cao và tỷ lệ nhóm phụ thuộc thấp. Điều này giúp tăng cường GDP và thúc đẩy sự phát triển kinh tế.\n\n2. Tăng trưởng GDP bình quân 5.9% trong mười năm qua: Việt Nam đã có mức tăng trưởng ấn tượng trong thập kỷ qua, và dự kiến sẽ tiếp tục phát triển mạnh mẽ trong năm 2023.\n\n3. Tăng trưởng trong ngành sản xuất: Sự tăng trưởng trong ngành sản xuất cũng đóng góp vào sự phát triển của nền kinh tế Việt Nam, với đầu tư ngày càng mạnh mẽ và các hiệp định thươ

In [5]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])
    print(token_count(value[1]), "tokens")
    print(value[1])

1122 tokens
Báo cáo thị trường tiêu dùng Việt Nam năm 2023: 9 nguyên nhân thúc đẩy sự tăng trưởng của Việt Nam trong thập kỷ tới

Báo cáo thị trường tiêu dùng Việt Nam năm 2023 đã đưa ra những nhận định quan trọng về xu hướng và kỳ vọng của người tiêu dùng Việt Nam trong thập kỷ tới. Dưới đây là 9 nguyên nhân chính thúc đẩy sự tăng trưởng của Việt Nam trong lĩnh vực tiêu dùng:

1. Dân số người đi làm, dân số lao động cao và tỷ lệ nhóm phụ thuộc thấp: Việt Nam sở hữu nguồn lực lao động lớn, với tỷ lệ người đi làm cao và tỷ lệ nhóm phụ thuộc thấp. Điều này giúp tăng cường GDP và thúc đẩy sự phát triển kinh tế.

2. Tăng trưởng GDP bình quân 5.9% trong mười năm qua: Việt Nam đã có mức tăng trưởng ấn tượng trong thập kỷ qua, và dự kiến sẽ tiếp tục phát triển mạnh mẽ trong năm 2023.

3. Tăng trưởng trong ngành sản xuất: Sự tăng trưởng trong ngành sản xuất cũng đóng góp vào sự phát triển của nền kinh tế Việt Nam, với đầu tư ngày càng mạnh mẽ và các hiệp định thương mại tự do.

4. Mức tiêu thụ

In [6]:
conclusion_function = plugin["Conclusion"]
summContent = " ".join([value[1] for value in values])
conclusion = await kernel.invoke(conclusion_function, sk.KernelArguments(conclusion = outline_dict["conclusion"], summContent = summContent))
print(str(conclusion))


Dựa trên các phân tích trong báo cáo, có thể kết luận rằng thị trường tiêu dùng Việt Nam trong năm 2023 đang chứng kiến sự phát triển mạnh mẽ và đầy tiềm năng. Với 9 nguyên nhân thúc đẩy tăng trưởng được đề cập, Việt Nam đang trở thành điểm đến hấp dẫn cho các doanh nghiệp muốn mở rộng hoạt động kinh doanh của mình. Sự tăng trưởng của GDP, lợi nhuận sản xuất, đầu tư, tiêu thụ hàng hóa và dịch vụ, du lịch, nền kinh tế internet, tài chính hộ gia đình, và hệ sinh thái tài chính đều đóng vai trò quan trọng trong việc thúc đẩy sự phát triển của thị trường tiêu dùng tại Việt Nam.

Với những xu hướng tích cực này, Việt Nam có tiềm năng phát triển mạnh mẽ trong lĩnh vực tiêu dùng trong thập kỷ tới. Để tồn tại và phát triển trên thị trường cạnh tranh của Việt Nam, các doanh nghiệp cần phải linh hoạt, sáng tạo và nắm bắt được những xu hướng và insights mới từ báo cáo thị trường tiêu dùng năm 2023. Chỉ thông qua việc hiểu rõ khách hàng và thị trường, họ mới có thể đưa ra các chiến lược hiệu quả v

In [7]:
print(token_count(str(conclusion)))

667


In [8]:
recommendation_function = plugin["Recommendation"]
summContent = " ".join([value[1] for value in values])
recommendation = await kernel.invoke(recommendation_function, sk.KernelArguments(recommendation = outline_dict["recommendations"], summContent = summContent))
print(str(recommendation))
print(token_count(str(recommendation)))

Dựa vào kết luận của báo cáo thị trường tiêu dùng năm 2023, chúng tôi đề xuất một số biện pháp cần thiết để tận dụng cơ hội và đối phó với thách thức trong thị trường tiêu dùng. Đầu tiên, các doanh nghiệp cần tập trung vào việc nâng cao chất lượng sản phẩm/dịch vụ để đáp ứng nhu cầu ngày càng cao của người tiêu dùng. Việc cải thiện trải nghiệm khách hàng, áp dụng công nghệ để tối ưu hóa hoạt động kinh doanh cũng là yếu tố quan trọng giúp doanh nghiệp vượt qua khó khăn, gia tăng hiệu quả và cạnh tranh trên thị trường.

Thứ hai, việc quảng cáo và tiếp cận thông minh thông qua các kênh trực tuyến cũng đóng vai trò quan trọng trong việc thu hút và giữ chân khách hàng. Sự linh hoạt và nhanh nhạy trong việc thích nghi với xu hướng tiêu dùng mới sẽ giúp các doanh nghiệp tồn tại và phát triển trong môi trường kinh doanh khó khăn.

Cuối cùng, việc hiểu rõ và đáp ứng được nhu cầu thay đổi của người tiêu dùng trong bối cảnh kinh tế - xã hội khó khăn là yếu tố quyết định sự thành công của các doan

In [9]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])

1122 tokens
Báo cáo thị trường tiêu dùng Việt Nam năm 2023: 9 nguyên nhân thúc đẩy sự tăng trưởng của Việt Nam trong thập kỷ tới

Báo cáo thị trường tiêu dùng Việt Nam năm 2023 đã đưa ra những nhận định quan trọng về xu hướng và kỳ vọng của người tiêu dùng Việt Nam trong thập kỷ tới. Dưới đây là 9 nguyên nhân chính thúc đẩy sự tăng trưởng của Việt Nam trong lĩnh vực tiêu dùng:

1. Dân số người đi làm, dân số lao động cao và tỷ lệ nhóm phụ thuộc thấp: Việt Nam sở hữu nguồn lực lao động lớn, với tỷ lệ người đi làm cao và tỷ lệ nhóm phụ thuộc thấp. Điều này giúp tăng cường GDP và thúc đẩy sự phát triển kinh tế.

2. Tăng trưởng GDP bình quân 5.9% trong mười năm qua: Việt Nam đã có mức tăng trưởng ấn tượng trong thập kỷ qua, và dự kiến sẽ tiếp tục phát triển mạnh mẽ trong năm 2023.

3. Tăng trưởng trong ngành sản xuất: Sự tăng trưởng trong ngành sản xuất cũng đóng góp vào sự phát triển của nền kinh tế Việt Nam, với đầu tư ngày càng mạnh mẽ và các hiệp định thương mại tự do.

4. Mức tiêu thụ